32

In [1]:
import sys
sys.path.append("../")
sys.path.append("../Modules/")

from Modules.simulation import Simulation
from Modules.cell_builder import SkeletonCell, CellBuilder
from Modules.constants import HayParameters

import os
from neuron import h

from logger import Logger

import numpy as np

--No graphics will be displayed.


In [2]:
sim = Simulation(SkeletonCell.Hay)
parameters = HayParameters(sim_name = 'test', h_tstop= 1000, all_synapses_off=True) # NOTE: synapses will crash the simulation since they are not mapped and will remain on deleted sections

In [3]:
%ls

 axial_currents.ipynb   builder_runtime.txt  'Figure 1.png'
 bAP_matching.ipynb     cell_graph.ipynb      reduction.ipynb


In [4]:
try:
    os.system("rm -r x86_64/")
except:
    pass

rm: cannot remove 'x86_64/': No such file or directory


In [5]:
%cd ../scripts

/home/drfrbc/Neural-Modeling/scripts


/home/drfrbc/miniconda3/envs/sim_env/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
os.system(f"nrnivmodl {sim.cell_type.value['modfiles']} > /dev/null 2>&1")

h.load_file('stdrun.hoc')
h.nrn_load_dll('./x86_64/.libs/libnrnmech.so')

1.0

In [7]:
# Build the cell
logger = Logger(None)
cell_builder = CellBuilder(sim.cell_type, parameters, logger)
cell, _ = cell_builder.build_cell()

(2024-04-26 14:24:02.089101)-[PID: 3969763]–[INFO]: Building SkeletonCell.Hay.
all: [L5PCtemplate[0].soma[0], L5PCtemplate[0].dend[0], L5PCtemplate[0].dend[1], L5PCtemplate[0].dend[2], L5PCtemplate[0].dend[3], L5PCtemplate[0].dend[4], L5PCtemplate[0].dend[5], L5PCtemplate[0].dend[6], L5PCtemplate[0].dend[7], L5PCtemplate[0].dend[8], L5PCtemplate[0].dend[9], L5PCtemplate[0].dend[10], L5PCtemplate[0].dend[11], L5PCtemplate[0].dend[12], L5PCtemplate[0].dend[13], L5PCtemplate[0].dend[14], L5PCtemplate[0].dend[15], L5PCtemplate[0].dend[16], L5PCtemplate[0].dend[17], L5PCtemplate[0].dend[18], L5PCtemplate[0].dend[19], L5PCtemplate[0].dend[20], L5PCtemplate[0].dend[21], L5PCtemplate[0].dend[22], L5PCtemplate[0].dend[23], L5PCtemplate[0].dend[24], L5PCtemplate[0].dend[25], L5PCtemplate[0].dend[26], L5PCtemplate[0].dend[27], L5PCtemplate[0].dend[28], L5PCtemplate[0].dend[29], L5PCtemplate[0].dend[30], L5PCtemplate[0].dend[31], L5PCtemplate[0].dend[32], L5PCtemplate[0].dend[33], L5PCtemplate[0].

In [8]:
from adjacency import get_all_descendants, find_terminal_descendants, get_branching_seg_indices

In [9]:
%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interactive, fixed, FloatSlider

# Modify the plot_segments function to use rotate_data
def plot_segments(seg_data, elevation=0, azimuth=-100, radius_scale=1.0, deleted_indices=[]):
    plt.close('all')  # Close any existing plots to prevent overload
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    for i, seg in enumerate(seg_data):
        x_points = [seg.coords['p0_0'].iloc[0], seg.coords['pc_0'].iloc[0], seg.coords['p1_0'].iloc[0]]
        y_points = [seg.coords['p0_1'].iloc[0], seg.coords['pc_1'].iloc[0], seg.coords['p1_1'].iloc[0]]
        z_points = [seg.coords['p0_2'].iloc[0], seg.coords['pc_2'].iloc[0], seg.coords['p1_2'].iloc[0]]
        
        radius = seg.coords['r'].iloc[0] * radius_scale  # Scale the radius for the linewidth
        color = 'red' if i in deleted_indices else 'black'
        radius = radius*10 if i in deleted_indices else radius
        ax.plot(x_points, z_points, y_points, linewidth=radius, color=color)

    ax.view_init(elev=elevation, azim=azimuth)
    ax.set_xlabel('X')
    ax.set_ylabel('Z')
    ax.set_zlabel('Y')
    plt.show()

In [10]:
from Modules.morph_reduction_utils import get_terminal_coordinates, create_segments_to_mech_vals, EXCLUDE_MECHANISMS, disconnect_root, reduce_subtree, calculate_nsegs_from_lambda, apply_params_to_section, create_seg_to_seg, copy_dendritic_mech, average_coordinates, calculate_third_point

In [11]:
adjacency_matrix = cell.compute_directed_adjacency_matrix()

In [12]:
from cell_model import find_nexus_seg
all_segments, seg_data = cell.get_segments(['all'])

nexus_seg_index = find_nexus_seg(cell, adjacency_matrix)
nexus_seg = all_segments[nexus_seg_index]
root_sections_to_reduce = nexus_seg.sec.children()
print(f"root_sections_to_reduce: {root_sections_to_reduce}")

root_sections_to_reduce: [L5PCtemplate[0].apic[50], L5PCtemplate[0].apic[37]]


In [13]:
def reduce_tree(root_section, all_segments, seg_data, adjacency_matrix):
    print(f"Reducing {root_section} and its descendants to single uniform cylindrical cable.")
    # gather the sectuibs and segments to delete.
    root_seg = root_section(0.000001)
    root_seg_index = all_segments.index(root_seg)
    root_seg_descendants_indices = get_all_descendants(adjacency_matrix, start_segment=root_seg_index, descendants=None)
    root_seg_descendants = [all_segments[seg_index] for seg_index in root_seg_descendants_indices]
    root_sec_descendants = list(np.unique([seg.sec for seg in root_seg_descendants]))
    sections_to_delete = np.unique([root_section] + root_sec_descendants) # root_sec will be in root_sec_descendants if root_sec is multiple segments
    deleted_seg_indices = [all_segments.index(seg) for sec in sections_to_delete for seg in sec]
    
    # get 3D coordinates # consider updating to get all the coordinates along the tree?
    terminal_descendant_indices = find_terminal_descendants(adjacency_matrix, start_index=root_seg_index, visited=None, terminal_indices=None)
    original_terminal_seg_coords = get_terminal_coordinates(seg_data, terminal_descendant_indices)
    original_root_seg_coords = seg_data[root_seg_index].coords # have to dissect this correctly
    new_section_begin_coordinates = [original_root_seg_coords.p0_0, original_root_seg_coords.p0_1, original_root_seg_coords.p0_2]
    
    # remove active conductances and get seg_to_mech dictionary that maps the holds mechanism values for the deleted segments
    deleted_segment_mechs_dict = create_segments_to_mech_vals(sections_to_delete=sections_to_delete, remove_mechs=False, exclude=EXCLUDE_MECHANISMS)
    
    # disconnet the root (ascendant/seg_to_reduce_from) from its ascendants. store x to reconnect later.
    root_parent_sec, root_connection_x_loc = disconnect_root(root_section)
    
    # get the new cable properties
    new_cable_properties = reduce_subtree(root_section)
    print(f"new_cable_properties: {new_cable_properties}")
    
    # determine nseg for the new branches
    new_cable_nseg = calculate_nsegs_from_lambda(new_cable_properties)
    
    # create reduced branches
    root_sec_type = str(root_section).split('.')[-1].split('[')[0]
    num_sec_of_root_sec_type = len(getattr(cell, root_sec_type))
    new_section_name  = 'New.' + root_sec_type + '[' + str(num_sec_of_root_sec_type)+ ']'
    new_section = h.Section(name=new_section_name)
    apply_params_to_section(new_section, new_cable_properties, new_cable_nseg)
    new_section.connect(root_parent_sec(root_connection_x_loc), 0)
    getattr(cell, root_sec_type).append(new_section)
    cell.all.append(new_section)
    
    # give 3D coordinates to new section
    average_terminal_coordinates = average_coordinates(original_terminal_seg_coords)
    new_section_end_coordinates = calculate_third_point(new_section_begin_coordinates, average_terminal_coordinates, new_cable_properties.length)
    new_section.pt3dadd(float(new_section_begin_coordinates[0].iloc[0]), float(new_section_begin_coordinates[1].iloc[0]), float(new_section_begin_coordinates[2].iloc[0]), new_section.diam)
    new_section.pt3dadd(float(new_section_end_coordinates[0].iloc[0]), float(new_section_end_coordinates[1].iloc[0]), float(new_section_end_coordinates[2].iloc[0]), new_section.diam)
    
    # deal with synapses

    # create segment to segment mapping
    reduction_frequency=0
    mapping_type = 'impedance'
    original_seg_to_reduced_seg, reduced_seg_to_original_seg = create_seg_to_seg(new_section,
                        root_sec_descendants,
                        root_section,
                        new_cable_properties,
                        mapping_type,
                        reduction_frequency)
    
    # copy active mechanisms
    copy_dendritic_mech(original_seg_to_reduced_seg=original_seg_to_reduced_seg,
                        reduced_seg_to_original_seg=reduced_seg_to_original_seg,
                        segment_to_mech_vals=deleted_segment_mechs_dict,
                        new_section=new_section,
                        mapping_type=mapping_type)
    
    # delete the sections to delete
    for sec in sections_to_delete:
        sec_type = str(sec).split('.')[-1].split('[')[0]
        getattr(cell,sec_type).remove(sec)
        cell.all.remove(sec)
        h.delete_section(sec=sec)
        
    return deleted_seg_indices, new_section

Equation (1):
$$
Z_{0,X} (\omega) = \frac{R_{\infty} \cosh (q (L - X))}{q \sinh(qL)},
$$

where

Equation (2):
$$
R_{\infty} = \frac{2}{\pi} \sqrt{R_m R_a} d^{-\frac{3}{2}},
$$

and

Equation (3):
$$
q = \sqrt{1 + i\omega\tau},
$$

where $\tau$ is the membrane time constant, $R_m C_m$.

From Eq. (1), the input impedance at $(X=0)$ is

Equation (4):
$$
Z_{0,0} (\omega) = \frac{R_{\infty}}{q} \coth(qL).
$$

We next want a cylindrical cable of electrotonic length $L$, in which both $|Z_{0,L} (\omega)|$ and $|Z_{0,0} (\omega)|$ are identical to those measured in the respective original stem dendrite (Fig. 1). For this purpose, we first look for an $L$ value in which the ratio $\frac{|Z_{0,L} (\omega)|}{|Z_{0,0} (\omega)|}$ is preserved. Dividing Eq. (1) by Eq. (4), we get


Equation (5):
$$
\frac{Z_{0,X} (\omega)}{Z_{0,0} (\omega)} = \frac{\cosh(q(L - X))}{\cosh(qL)},
$$

which can be expressed as

Equation (6):
$$
\frac{Z_{0,X} (\omega)}{Z_{0,0} (\omega)} = \frac{\cosh (a(L - X) + i b(L - X))}{\cosh(aL + i bL)} = M \exp(i \phi),
$$


where $a$ and $b$ are the real and the imaginary parts of $q$, respectively, and $M$ and $phi$ are the modulus and phase angle of this complex ratio.

As shown previously\(^{62}\), it follows that

$$
M = \frac{|Z_{0,X} (\omega)|}{|Z_{0,0} (\omega)|} = \left[ \frac{\cosh(2a (L - X)) + \cos(2b (L - X))}{\cosh(2aL) + \cos(2bL)} \right]^{0.5}
$$

and

$$
\phi = \arctan \left[ \frac{\tanh(a (L - X)) \tan(b (L - X))}{1 - \tanh(a (L - X)) \tan(b (L - X))} \right]
$$

Importantly, for a fixed $( M )$ (and a given $( \omega )$) there is a unique value of $( L )$ that satisfies Eq. (7) (see Fig. 4 in ref. $( ^{62} )$ and note the one-to-one mapping between $( M )$ and $( L )$ for a given $( \omega )$ value). However, there are an infinite number of cylindrical cables (with different diameters and lengths) that have identical $( L )$ values preserving a given $( M )$ value in Eq. (7).


We next need a unique cable, with a specific diameter $d$, that also preserves the measured value of $|Z_{0,0}(\omega)|$ (and therefore it also preserves $Z_{0,L}(\omega)$, see Eq. (7)).

From Eqs. (2) and (4) we get

$$
Z_{0,0} (\omega) = \frac{2}{\pi q} \sqrt{R_m R_a} d^{-\frac{3}{2}} \coth(qL)
$$

and thus

$$
|Z_{0,0} (\omega)| = \left| \frac{2}{\pi q} \sqrt{R_m R_a} d^{-\frac{3}{2}} \coth(qL) \right|
$$

from which we compute the diameter, $d$, for that cylinder

$$
|d| = \left( \frac{2}{\pi} \sqrt{R_m R_a} \frac{\coth(qL)}{|Z_{0,0} (\omega)|} \right)^{\frac{2}{3}}.
$$


Equations (1)--(11) provide the unique cylindrical cable (with a specific $d$ and $L$, and the given membrane and axial properties) that preserves the values of $|Z_{0,L} (\omega)|$ and $|Z_{0,0} (\omega)|$ as in the respective stem dendrite. Note that this unique cable does not necessarily preserve the phase ratio ($\phi$ in Eq. (8)) as in the original tree.


In [14]:
all_deleted_seg_indices = []
new_sections = []
for root_section in root_sections_to_reduce:
    deleted_seg_indices, new_section = reduce_tree(root_section, all_segments, seg_data, adjacency_matrix)
    all_deleted_seg_indices.extend(deleted_seg_indices)
    new_sections.append(new_section)

Reducing L5PCtemplate[0].apic[50] and its descendants to single uniform cylindrical cable.
new_cable_properties: CableParams(length=1140.2024431017408, diam=1.6538087750516277, space_const=837.8282558822426, cm=2.0, rm=16977.92869269949, ra=100.0, e_pas=-90.0, electrotonic_length=1.3609023509252438)
Reducing L5PCtemplate[0].apic[37] and its descendants to single uniform cylindrical cable.
new_cable_properties: CableParams(length=702.1941510897877, diam=1.4055829093126275, space_const=772.397022359403, cm=2.0, rm=16977.92869269949, ra=100.0, e_pas=-90.0, electrotonic_length=0.9091103807531908)


In [15]:
new_section

New.apic[81]

Show Reduced Cell

In [16]:
# Now include the updated plot_segments function in your interactive widget setup
interactive_plot = interactive(plot_segments, 
                               seg_data=fixed(seg_data),
                               elevation=(-90, 90, 1),
                               azimuth=(-180, 180, 1),
                               radius_scale=FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0),
                               deleted_indices=fixed(all_deleted_seg_indices))
interactive_plot

interactive(children=(IntSlider(value=0, description='elevation', max=90, min=-90), IntSlider(value=-100, desc…

In [17]:
# new_section.name()
cell.all[-3:]

[L5PCtemplate[0].axon[0], New.apic[109], New.apic[81]]

In [18]:
new_segments = [list(new_section) for new_section in new_sections]

In [19]:
cell.apic[-3:]

[L5PCtemplate[0].apic[108], New.apic[109], New.apic[81]]

In [20]:
all_segments_after_reduction, new_seg_data = cell.get_segments(['all'])

In [21]:
new_seg_indices = []
for seg in new_segments: # new_sections is a list of segments as of writing this
    new_seg_indices.append(all_segments_after_reduction.index(seg))

ValueError: [New.apic[109](0.0384615), New.apic[109](0.115385), New.apic[109](0.192308), New.apic[109](0.269231), New.apic[109](0.346154), New.apic[109](0.423077), New.apic[109](0.5), New.apic[109](0.576923), New.apic[109](0.653846), New.apic[109](0.730769), New.apic[109](0.807692), New.apic[109](0.884615), New.apic[109](0.961538)] is not in list

In [ ]:
# Now include the updated plot_segments function in your interactive widget setup
interactive_plot = interactive(plot_segments, 
                               seg_data=fixed(new_seg_data),
                               elevation=(-90, 90, 1),
                               azimuth=(-180, 180, 1),
                               radius_scale=FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0),
                               deleted_indices=fixed(new_seg_indices))
interactive_plot

In [ ]:
parameters

In [ ]:
os.mkdir(sim.path)

In [ ]:
parameters.path = os.path.join(sim.path, parameters.sim_name)

In [ ]:
parameters.path

In [ ]:
sim.set_all_recorders(cell, parameters)

In [ ]:
parameters.record_all_v

In [ ]:
sim.simulate(cell, parameters)

In [ ]:
import pickle
with open(os.path.join(parameters.path, "parameters.pickle"), "wb") as file:
    pickle.dump(parameters, file)

In [ ]:
import analysis
soma_spikes = analysis.DataReader.read_data(parameters.path, "soma_spikes")

In [ ]:
v = analysis.DataReader.read_data(parameters.path, "v")

In [ ]:
v[0]